# Dynamic Prompt
<img src="./assets/LC_DynamicPrompts.png" width="500">

## Setup

Load and/or check for needed environmental variables

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env("example.env")

DASHSCOPE_API_KEY=****7258
LANGSMITH_API_KEY=****4e15
LANGSMITH_TRACING=true
LANGSMITH_PROJECT=****ials


In [2]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [3]:
from dataclasses import dataclass


@dataclass
class RuntimeContext:
    is_employee: bool
    db: SQLDatabase

In [4]:
from langchain_core.tools import tool
from langgraph.runtime import get_runtime

@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    runtime = get_runtime(RuntimeContext)
    db = runtime.context.db

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [5]:
SYSTEM_PROMPT_TEMPLATE = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
{table_limits}
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

## Build a Dynamic Prompt
Utilize runtime context and middleware to generate a dynamic prompt.

In [6]:
from langchain.agents.middleware.types import ModelRequest, dynamic_prompt


@dynamic_prompt
def dynamic_system_prompt(request: ModelRequest) -> str:
    if not request.runtime.context.is_employee:
        table_limits = "- Limit access to these tables: Album, Artist, Genre, Playlist, PlaylistTrack, Track."
    else:
        table_limits = ""

    return SYSTEM_PROMPT_TEMPLATE.format(table_limits=table_limits)

Include middleware in `create_agent`.

In [7]:
from langchain.agents import create_agent
from langchain_community.chat_models import ChatTongyi

# Initialize Qwen model via DashScope (Alibaba Cloud)
model = ChatTongyi(model="qwen-turbo")

agent = create_agent(
    model=model,
    tools=[execute_sql],
    middleware=[dynamic_system_prompt],
    context_schema=RuntimeContext,
)

In [8]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=False, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================

To determine the most costly purchase by Frank Harris, we need to look at the purchases made by him. However, there is no table that directly records purchases or transaction details in the provided schema.

If you have more information or a specific table that tracks purchases, please provide it so I can assist further. Otherwise, with the current schema, it's not possible to determine the most costly purchase by Frank Harris.


In [9]:
question = "What is the most costly purchase by Frank Harris?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context=RuntimeContext(is_employee=True, db=db),
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is the most costly purchase by Frank Harris?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_e70ef2e0ca0549c1bc883b)
 Call ID: call_e70ef2e0ca0549c1bc883b
  Args:
    query: SELECT MAX(purchase_amount) FROM purchases WHERE buyer_name = 'Frank Harris';
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: purchases
[SQL: SELECT MAX(purchase_amount) FROM purchases WHERE buyer_name = 'Frank Harris';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================

It seems there is no table named `purchases` in the database. Let me check if there are any other tables that might contain purchase-related information.
Tool Calls:
  execute_sql (call_f2fd405ae2a049bc8990a8)